# Testing nanoLLM from NVidia

Based on this tutorial: https://www.jetson-ai-lab.com/tutorial_nano-llm.html

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "distilgpt2"  # Use the same model as the chat code
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Dummy input (required for ONNX export)
dummy_input = tokenizer("Hello world!", return_tensors="pt")

# Export to ONNX format
torch.onnx.export(
    model, 
    (dummy_input["input_ids"],),
    "distilgpt2.onnx",  # Match the ONNX file name to the model
    input_names=["input_ids"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {1: "sequence_length"},
        "logits": {1: "sequence_length"}
    },
    opset_version=14
)

/home/miroslaw/.local/lib/python3.8/site-packages/transformers/modeling_attn_mask_utils.py:88: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
/home/miroslaw/.local/lib/python3.8/site-packages/transformers/modeling_attn_mask_utils.py:164: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:


In [ ]:
import onnxruntime as ort
from transformers import AutoTokenizer
import numpy as np

# Load ONNX model
session = ort.InferenceSession("distilgpt2.onnx")

# Prepare tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Encode input
inputs = tokenizer("The weather is ", return_tensors="np")

# Run inference
ort_inputs = {'input_ids': inputs["input_ids"]}
logits = session.run(["logits"], ort_inputs)[0]

# Generate next token
next_token_id = np.argmax(logits[0, -1, :])
next_token = tokenizer.decode([next_token_id])

print("Next token predicted:", next_token)

Next token predicted: 's


## Now a chat model

Please note that we need to change the model to be a bit bigger, because the previous one does not really provide a good chat.

In [1]:
# --- Export to ONNX (update model and directory) ---
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.onnxruntime import ORTModelForCausalLM
import torch

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

dummy_input = tokenizer("Hello world!", return_tensors="pt")
torch.onnx.export(
    model, 
    (dummy_input["input_ids"],),
    "distilgpt2.onnx",
    input_names=["input_ids"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {1: "sequence_length"},
        "logits": {1: "sequence_length"}
    },
    opset_version=14)

model_id = "distilgpt2"
save_directory = "./distilgpt2-chat-onnx-simple"

ort_model = ORTModelForCausalLM.from_pretrained(
    model_id,
    export=True,
    use_cache=False,
    use_io_binding=False)
tokenizer.save_pretrained(save_directory)
ort_model.save_pretrained(save_directory)

ModuleNotFoundError: No module named 'optimum'

In [ ]:
import onnxruntime as ort
from transformers import AutoTokenizer
import numpy as np

model_dir = "./distilgpt2-chat-onnx-simple"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
session = ort.InferenceSession(f"{model_dir}/model.onnx")

def generate_response(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="np")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    for _ in range(max_new_tokens):
        position_ids = np.arange(input_ids.shape[1]).reshape(1, -1).astype(np.int64)
        ort_inputs = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "position_ids": position_ids
        }
        logits = session.run(None, ort_inputs)[0]
        next_token_logits = logits[:, -1, :]
        next_token_id = np.argmax(next_token_logits, axis=-1).reshape(1, 1)
        input_ids = np.concatenate([input_ids, next_token_id], axis=-1)
        attention_mask = np.concatenate(
            [attention_mask, np.ones((attention_mask.shape[0], 1), dtype=np.int64)], 
            axis=-1
        )
        if next_token_id[0, 0] == tokenizer.eos_token_id:
            break

    return tokenizer.decode(input_ids[0], skip_special_tokens=True)

prompt = "User:Who is Fibonacci?\nAssistant:"
response = generate_response(prompt)
print(response)

User:Show example of code?
Assistant:Show example of code?
Assistant:Show example of code?
Assistant:Show example of code?
Assistant:Show example of code?
Assistant:Show example of code?
Assistant:Show example of code?
Assistant:Show example
